In [4]:
import tarfile
from pathlib import Path
from io import BytesIO

import numpy as np
import pandas as pd
import fastai

from utils.prepare_data import read_data, save_data, target_columns

In [1]:
# tar = Path('../../output/simulation-data/recombination-lower/data.tar')
# tarfile.is_tarfile(tar)

# with tarfile.open(tar) as t:
#     names = t.getnames()
#     array_file = BytesIO()
#     array_file.write(t.extractfile(names[0]).read())
#     array_file.seek(0)
#     a = np.load(array_file)

NameError: name 'Path' is not defined

In [5]:
x = pd.DataFrame({'epoch': [1, 2, 3, 4], 'loss': [16, 8, 4, 2]})

In [7]:
# Toy output files
from pathlib import Path

save_data(x, snakemake.output["fit_report"])
if snakemake.params["save_model"]:
    Path(snakemake.output["fit_model"]).touch()
if snakemake.params["save_inferences"]:
    Path(snakemake.output["training_inferences"]).touch()
    Path(snakemake.output["validation_inferences"]).touch()